# This notebook is to downlod user tweets

In [5]:
# Twitter tweet download to call API
from twitter import *

In [8]:
import pandas as pd
import json
from os.path import join
from time import time
from tqdm import tqdm_notebook as tqdm

import csv
import numpy as np

import preprocessor # Twitter pre-processor

# language detection
import spacy
from spacy_langdetect import LanguageDetector
from time import time

In [85]:
# API a/c details

key = ""
secret = ""
token=""

In [86]:
# Creating Twitter object to download the tweets

twitter = Twitter(auth = OAuth("",
                  "",
                  "",
                  ""))

In [87]:
user_tweets_map = {}
twitter_errors = []
total_request_made = 0
total_request_limit = 90000

In [88]:
# download user tweets using twtter object

def get_all_tweets(user_id, total_request_made):
    total_results = []
    results = twitter.statuses.user_timeline(user_id = user_id, count = 200)
    total_results.extend(results)
    if len(results) == 0:
        return total_results
    
    last_id = results[-1]['id']

    while last_id != None:
        try:
            results = twitter.statuses.user_timeline(user_id = user_id, count = 200, max_id = last_id)
            total_request_made += 1
        except TwitterHTTPError as error:
            twitter_errors.append(error)
        
        if len(results) == 0:
            last_id = None
        elif last_id == results[-1]['id']:
            last_id = None
        else:
            last_id = results[-1]['id']
            total_results.extend(results)
    
    return total_results

In [89]:
genders=['male','female']

In [90]:
# Creating dataset

def get_user_tweets():
    t0 = time()
    csv_path = "C:\\Users\\pvashisth\\Desktop\\ai_course\\Git\\twitter-gender-classification\\src\\data.csv"
    
    df = pd.read_csv(csv_path, encoding='latin-1')
    
    df = df.loc[(df['gender'].isin(genders)) & (df['gender:confidence'] == 1)]
    print(df.shape)
    
    for index, row in tqdm(df.iterrows()):
        if total_request_made >= total_request_limit:
            print("Stopping for today!!!!")
            print("Next user to be processed is {}: ".format(row['user_id']))
            return
        try:
            results = get_all_tweets(row['user_id'], total_request_made)
        except TwitterHTTPError as error:
            twitter_errors.append(error)
        
        user_tweets_map[row['user_id']] = {
            'gender': row['gender'],
            'results': results
        }
    
    # save the dict.
    with open(join('C:\\Users\\pvashisth\\Desktop\\ai_course\\Git\\twitter-gender-classification\\src', 'user_all_tweets_data_part_1'), 'w', encoding='utf-8') as w:
        json.dump(user_tweets_map, w)
    
    print("Completed in {}".format(time() - t0))

In [91]:
get_user_tweets()

(10020, 26)


Completed in 3792.848995447159


In [38]:
len(user_tweets_map)

10020

In [39]:
type(user_tweets_map)

dict

### Per user tweet Analysis

In [7]:
total_tweets = []
for utm in user_tweets_map:
    total_tweets.append(len(user_tweets_map[utm]['results']))

In [8]:
len(total_tweets)

10020

In [9]:
# data analysis of per user tweet

np.mean(total_tweets), np.median(total_tweets), np.std(total_tweets), np.min(total_tweets), np.max(total_tweets)

(34.56217564870259, 0.0, 272.40745141659124, 0, 3218)

## Data transformation (cleaning, data analysis- creating dataset to train ML model)

In [9]:
user_tweets_map = {}

In [12]:
with open("C:\\Users\\pvashisth\\Desktop\\ai_course\\Git\\twitter-gender-classification\\src\\user_all_tweets_data_part_1", 'r') as r:
    user_tweets_map = json.loads(r.read())

In [13]:
len(user_tweets_map)

10020

In [14]:
len(user_tweets_map['815719226']['results'][0]['text'].split())

14

### Tweets Length Analysis

In [15]:
tweets_lengths = []
for user_id in user_tweets_map:
    tweets_data = user_tweets_map[user_id]['results']
    
    for tweet in tweets_data:
        tweets_lengths.append(len(tweet['text'].split())) # text property contains the acutal tweet

In [16]:
len(tweets_lengths)

346313

In [17]:
np.mean(tweets_lengths), np.median(tweets_lengths), np.std(tweets_lengths), np.min(tweets_lengths), np.max(tweets_lengths)

(12.082344584234493, 11.0, 6.738272884423014, 1, 53)

### Filtering tweets by words count

In [18]:
more_than_50 = [t for t in tweets_lengths if t >=50]
more_than_40 = [t for t in tweets_lengths if t >=40]
more_than_30 = [t for t in tweets_lengths if t >=30]
more_than_20 = [t for t in tweets_lengths if t >=20]
more_than_10 = [t for t in tweets_lengths if t >=10]
more_than_5 = [t for t in tweets_lengths if t >=5]

len(more_than_50), len(more_than_40), len(more_than_30), len(more_than_20), len(more_than_10), len(more_than_5)

(14, 27, 1065, 58183, 199659, 302549)

In [19]:
dataset = {}
dataset['male'] = []
dataset['female'] = []

### Tweets cleaning using the python package 'Pre-Processor'

In [20]:
def get_tweets_for_user(results):
    tweets = []
    for tweet_entry in results:
        tweet = tweet_entry['text']
        tweet = preprocessor.clean(tweet)
        
        tweets.append(tweet)
    
    return tweets

In [21]:
for user_id in user_tweets_map:
    gender = user_tweets_map[user_id]['gender']
    
    if gender == 'male':
        dataset['male'].extend(get_tweets_for_user(user_tweets_map[user_id]['results']))
    elif gender == 'female':
        dataset['female'].extend(get_tweets_for_user(user_tweets_map[user_id]['results']))
    else:
        raise Exception("blah blah")

In [22]:
len(dataset['male']), len(dataset['female'])

(163655, 182658)

In [23]:
male_tweets = dataset['male']
female_tweets = dataset['female']

In [ ]:
# Male tweets

In [24]:
more_than_50 = [t for t in male_tweets if len(t.split()) >=50]
more_than_40 = [t for t in male_tweets if len(t.split()) >=40]
more_than_30 = [t for t in male_tweets if len(t.split()) >=30]
m_more_than_20 = [t for t in male_tweets if len(t.split()) >=20]
more_than_10 = [t for t in male_tweets if len(t.split()) >=10]
more_than_5 = [t for t in male_tweets if len(t.split()) >=5]

len(more_than_50), len(more_than_40), len(more_than_30), len(m_more_than_20), len(more_than_10), len(more_than_5)

(0, 5, 274, 19387, 80620, 132497)

In [25]:
male_tweets[4]

': My story, please go for your smears ladies BR - News - Cervical Cancer survivor tell her story'

In [ ]:
# Female tweets

In [26]:
more_than_50 = [t for t in female_tweets if len(t.split()) >=50]
more_than_40 = [t for t in female_tweets if len(t.split()) >=40]
more_than_30 = [t for t in female_tweets if len(t.split()) >=30]
f_more_than_20 = [t for t in female_tweets if len(t.split()) >=20]
more_than_10 = [t for t in female_tweets if len(t.split()) >=10]
more_than_5 = [t for t in female_tweets if len(t.split()) >=5]

len(more_than_50), len(more_than_40), len(more_than_30), len(f_more_than_20), len(more_than_10), len(more_than_5)

(0, 7, 384, 22545, 93416, 148373)

## Language detection

In [27]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [28]:
t0 = time()
male_english_tweets = []
for mt in m_more_than_20:
    doc = nlp(mt)
    
    if doc._.language['language'] == 'en':
        male_english_tweets.append(mt)
    
print(len(male_english_tweets))
print(time() - t0)

9762
278.7980568408966


In [29]:
t0 = time()
female_english_tweets = []
for ft in f_more_than_20:
    doc = nlp(ft)
    
    if doc._.language['language'] == 'en':
        female_english_tweets.append(ft)
    
print(len(female_english_tweets))
print(time() - t0)

11259
341.67352747917175


In [30]:
len(male_english_tweets), len(female_english_tweets)

(9762, 11259)

### Save the english tweets as tsv and files

In [31]:
for index, f in enumerate(male_english_tweets):
    with open(join("C:\\Users\\pvashisth\\Desktop\\ai_course\\Git\\twitter-gender-classification\\src\\Data\\pos", str(index) + ".txt"), 'w', encoding='utf-8') as w:
        w.write(f)

In [32]:
for index, f in enumerate(female_english_tweets):
    with open(join("C:\\Users\\pvashisth\\Desktop\\ai_course\\Git\\twitter-gender-classification\\src\\Data\\neg", str(index) + ".txt"), 'w', encoding='utf-8') as w:
        w.write(f)

## END